In [ ]:
!cp -r /kaggle/input/save-sentence-transformer-repo/sentence-transformers /tmp/sentence-transformers
!pip install /tmp/sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

modelPath = "/kaggle/input/save-sentence-transformer-repo/bert-base/"

model = SentenceTransformer(modelPath)

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import pickle

In [ ]:
NFOLDs = 5

In [ ]:
dataset_test = pd.read_csv('/kaggle/input/feedback-prize-effectiveness/test.csv')

In [ ]:
dataset_test['text'] = dataset_test['essay_id'].apply(lambda x: open(f'/kaggle/input/feedback-prize-effectiveness/test/{x}.txt').read())

In [ ]:
preds = []

In [ ]:
dataset_test["discourse_text_encoding"] = model.encode(dataset_test["discourse_text"].tolist(),show_progress_bar=True, batch_size=256).tolist()

In [ ]:
dataset_test["text_encoding"] = model.encode(dataset_test["text"].tolist(),show_progress_bar=True, batch_size=256).tolist()

In [ ]:
dataset_test.head()

In [ ]:
test_discourse_text =  dataset_test["discourse_text_encoding"].tolist()
test_text = dataset_test["text_encoding"].tolist()

In [ ]:
for fold in range(NFOLDs):    
    filename = f"../input/train-logisticregression-fpe-sentencetransformer/ohe_{fold}.pkl"
    with open(filename, 'rb') as f:
        ohe = pickle.load(f)
        
    filename = f"../input/train-logisticregression-fpe-sentencetransformer/clf_{fold}.pkl"
    with open(filename, 'rb') as f:
        clf = pickle.load(f)
        
    te_type_ohe =  sparse.csr_matrix(ohe.transform(dataset_test["discourse_type"].values.reshape(-1,1)))
    
    test_dat = sparse.hstack((te_type_ohe,test_discourse_text,test_text))
    preds.append(clf.predict_proba(test_dat))

In [ ]:
dataset_submission = pd.read_csv("../input/feedback-prize-effectiveness/sample_submission.csv")

In [ ]:
all_preds = np.array(preds).mean(0)
print(all_preds.shape)
dataset_submission.loc[:,"Ineffective"] = all_preds[:,0]
dataset_submission.loc[:,"Adequate"] = all_preds[:,1]
dataset_submission.loc[:,"Effective"] = all_preds[:,2]
dataset_submission.head()

In [ ]:
dataset_submission.to_csv('submission.csv',index=False)